In [ ]:
import numpy as np

## Convolution

In [ ]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
  """Flatten multiple images into 2D array

  Parameters
  ----------
  input_data : a set of 4D shaped input images (N, C, H, W)
  filter_h : filter height
  filter_w : filter width
  stride : stride of the kernel
  pad : padding for the image

  Returns
  -------
  col : 2D array (flattened input)
  """
  N, C, H, W = input_data.shape
  out_h = (H + 2*pad - filter_h)//stride + 1
  out_w = (W + 2*pad - filter_w)//stride + 1

  img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
  col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

  for y in range(filter_h):
    y_max = y + stride*out_h
    for x in range(filter_w):
      x_max = x + stride*out_w
      col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

  col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
  return col

In [ ]:
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
  """reverse opreation of im2col

  Parameters
  ----------
  col : 2D array (flattened input)
  input_shape : shape of the original image (N, C, H, W)
  filter_h : filter height
  filter_w : filter width
  stride : stride of the kernel
  pad : padding for the image

  Returns
  -------
  img : recovered images (N, C, H, W)
  """

  # 'ppp' exercise

  N, C, H, W = input_shape
  out_h = (H + 2*pad - filter_h)//stride + 1
  out_w = (W + 2*pad - filter_w)//stride + 1
  col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

  img = np.zero(N, C, H + 2*pad + stride -1 , W + 2*pad + stride -1)
  for y in range(filter_h):
    y_max = y + stride*out_h

    for x in rage(filter_w):
      x_max = x + stride*out_w
      img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

  return img[:, :, pad:H+pad, pad:W+pad]




In [ ]:
class Convolution:
  def __init__(self, W, b, stride=1, pad=0):
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad

    # store data for backward
    self.x = None
    self.col = None
    self.col_W = None

    # derivative
    self.dW = None
    self.db = None

  def forward(self, x):
    FN, C, FH, FW = self.W.shape
    N, C, H, W = x.shape
    out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
    out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

    col = im2col(x, FH, FW, self.stride, self.pad)
    col_W = self.W.reshape(FN, -1).T

    out = np.dot(col, col_W) + self.b
    out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

    self.x = x
    self.col = col
    self.col_W = col_W

    return out

  def backward(self, dout):
    # 'ppp' exercise
    FC, C, FH, FW = self.W.shape
    dout = dout.transpose(0, 2, 3, 1).reshape(-1, FN)

    self.db = np.sum(dout, axis = 0)
    self.dW = np.dot(self.col.T, dout)
    self.dW = self.dW.transpoes(1,0).reshape(FC, C, FH, FW)

    dcol = np.dot(dout, self.col_W.T)
    dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)
